In [16]:
!pip3 install nltk

  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.9 MB/s eta 0:00:00a 0:00:010m
Using cached click-8.1.7-py3-none-any.whl (97 kB)


In [50]:
import os
import glob
def extract_text_from_folder(folder_path):
    text_data = []
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith(('.txt', '.rst')):
                file_path = os.path.join(root, file_name)
                with open(file_path, 'r', encoding='utf-8') as file:
                    text = file.read()
                    text_data.append(text)
    return text_data

In [39]:
text_data = extract_text_from_folder("./raw_dataset_USFTHF/docs/")


index.rst
index.rst
networking.rst
gcc.rst
system.rst
storage.rst
peripherals.rst
ieee802154.rst
index.rst
build-system.rst
networking.rst
gcc.rst
tools.rst
system.rst
bluetooth-classic.rst
removed-components.rst
protocols.rst
storage.rst
peripherals.rst
provisioning.rst
bluetooth-low-energy.rst
index.rst
wifi.rst
gcc.rst
system.rst
bluetooth-classic.rst
protocols.rst
storage.rst
peripherals.rst
ieee802154.rst
index.rst
system.rst
storage.rst
peripherals.rst
bluetooth-low-energy.rst
security.rst
index.rst
esp32c3_log.inc
esp32c2_log.inc
esp32h2_log.inc
esp32_log.inc
host-based-security-workflows.rst
secure-boot-v1.rst
secure-boot-v2.rst
flash-encryption.rst
esp32c5_log.inc
esp32p4_log.inc
esp32s2_log.inc
esp32s3_log.inc
esp32c6_log.inc
security.rst
index.rst
error-codes.rst
kconfig.rst
template.rst
api-conventions.rst
index.rst
esp_eth.rst
esp_dpp.rst
esp_wifi.rst
esp_now.rst
esp_smartconfig.rst
esp-wifi-mesh.rst
esp_nan.rst
esp_netif.rst
esp_openthread.rst
esp_netif_driver.rst
index.r

In [51]:
text_data

['Migration Guides\n*****************\n\n:link_to_translation:`zh_CN:[中文]`\n\nESP-IDF 5.x Migration Guide\n===========================\n\n.. toctree::\n    :maxdepth: 1\n\n    release-5.x/5.0/index\n    release-5.x/5.1/index\n    release-5.x/5.2/index\n    release-5.x/5.3/index\n',
 'Migration from 5.0 to 5.1\n-------------------------\n\n:link_to_translation:`zh_CN:[中文]`\n\n.. toctree::\n    :maxdepth: 1\n\n    gcc\n    :SOC_IEEE802154_SUPPORTED: ieee802154\n    peripherals\n    storage\n    networking\n    system\n',
 'Networking\n===========\n\n:link_to_translation:`zh_CN:[中文]`\n\nSNTP\n----\n\nSNTP module now provides thread safe APIs to access lwIP functionality. It is recommended to use :doc:`ESP_NETIF </api-reference/network/esp_netif>` API. Please refer to the chapter :ref:`esp_netif-sntp-api` for more details.\n',
 "GCC\n***\n\n:link_to_translation:`zh_CN:[中文]`\n\n\nGCC Version\n===========\n\nThe previous GCC version was GCC 11.2.0. This has now been upgraded to GCC 12.2.0 on

In [52]:
import pandas as pd
import numpy as np
try:
    text_data_df = pd.DataFrame(np.array(text_data), columns=["text_data"])
except:
    text_data_numpy = np.empty(len(text_data), dtype=object)
    text_data_numpy[:] = text_data
    text_data_df = pd.DataFrame(np.array(text_data_numpy, dtype=object), columns=["text_data"])

In [76]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
# Drop duplicate rows across all columns
text_data_df = text_data_df.drop_duplicates()
train_data, test_data = train_test_split(text_data_df, test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_data)
eval_dataset = Dataset.from_pandas(test_data)

In [77]:

train_dataset,eval_dataset

(Dataset({
     features: ['text_data', '__index_level_0__'],
     num_rows: 289
 }),
 Dataset({
     features: ['text_data', '__index_level_0__'],
     num_rows: 73
 }))

In [73]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
token_count=0;
for text in text_data:
    tokens = word_tokenize(text)
    token_count += len(tokens)
print(token_count)

[nltk_data] Downloading package punkt to /Users/biboxdev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


693388


In [98]:
def formatting_func(example):
    text = f"### The following is a note by Eevee the Dog: {example['text_data']}"
    return text

In [101]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

def generate_and_tokenize_prompt(prompt):
    # print(prompt)
    return tokenizer(formatting_func(prompt))

In [102]:

tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

Map: 100%|██████████| 289/289 [00:01<00:00, 217.03 examples/s]
